# Re-Sampling of OPM Data and saving it to a .fif file

In [ ]:
#Import necessary libraries and modules
import mne
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import array
import os
import sys

#### Checking if the files are accessible

In [ ]:
# List contents of the directory
directory_path = '/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings'
files = os.listdir(directory_path)
file_name='Rec1.fif'
data_path=os.path.join(directory_path,file_name)
# Check if the fif file exists in the location
if not os.path.isfile(data_path):
    raise FileNotFoundError(f"The file {data_path} does not exist.")
file_exists = os.path.isfile(data_path)
print(f"File exists: {file_exists}")
print(f"Loading data from {data_path}...")

#### Data Loading

In [ ]:
#Data Loading
raw = mne.io.read_raw_fif(data_path, preload=True);
# Detailed information about the data
#print(raw.info);

#### Filtering and Resampling

In [ ]:
# Provide your desired resampling frequency.
desired_sfreq = 1000  
output_file_path = None  

current_sfreq = raw.info['sfreq']
print(f"Current sampling frequency: {current_sfreq} Hz")

# Checking if resample can be done
if current_sfreq == desired_sfreq:
    print(f"The data is already at the desired sampling frequency of {desired_sfreq} Hz.")
    sys.exit(0)

# Apply a low-pass filter at one-fourth of the desired sampling frequency to avoid aliasing
lowpass_freq = desired_sfreq / 4.0
print(f"Applying low-pass filter with cutoff frequency of {lowpass_freq} Hz...")
raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

# Resample the data using MNE default resampling function
print(f"Resampling data to {desired_sfreq} Hz...")
raw_filtered.resample(sfreq=desired_sfreq)

# Set the output file path
if output_file_path is None:
    output_file_path = data_path.replace('.fif', f'_resampled_{desired_sfreq}Hz.fif')

# Save the resampled data to a new .fif file
print(f"Saving resampled data to {output_file_path}...")
raw_filtered.save(output_file_path, overwrite=True)

print("Resampling complete.")



In [8]:
print(raw_filtered.info)

<Info | 16 non-empty values
 bads: []
 ch_names: s1_bz, L209_bz-s65, s2_bz, L207_bz-s66, R204_bz-s3, L205_bz-s67, ...
 chs: 72 Magnetometers, 1 Stimulus
 custom_ref_applied: False
 description: {"chassis":{"version":"0.7.7- ...
 experimenter: AR
 file_id: 4 items (dict)
 gantry_angle: 0.0
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 250.0 Hz
 meas_date: 2024-08-23 15:22:40 UTC
 meas_id: 4 items (dict)
 nchan: 73
 proj_id: 1 item (ndarray)
 proj_name: NoiseDetection
 projs: []
 sfreq: 1000.0 Hz
 xplotter_layout: None
>
